<H1 style=text-align:center;><b>Projet Maths-Info : Traveling Salesman Problem</b></H1>
<h1 style=text-align:center;>(AMU) EADS 2024-2025</h1>

<h2 style=text-align:center;><i>Projet présenté par : Alexandre ROMANO / Victoria BOUCHET / Vahé TILDIAN</i></h2>

<br><br><br>

## <u>Introduction</u>

Le <strong>Problème du Voyageur de Commerce</strong>, abrégé en TSP, est un problème d'optimisation combinatoire assez emblématique dans le monde de l'informatique théorique.

Il consiste à déterminer un chemin de longueur minimale passant exactement une fois par ville et revenant au point de départ, dans un réseau de $n$ villes. 

Cette approche est $NP$-difficile, ce qui signifie qu'aujourd'hui, aucune approche existante permet d'obtenir la solution optimale à ce problème. 
Ce qui le rend emblématique est sa difficulté de résolution malgré un énoncé très simple.

$\fbox{Le Projet}$

Notre projet consiste à explorer plusieurs <strong>heuristiques</strong> <i>(ce sont des méthodes permettant d'obtenir une solution faisable, une résolution approximative donc pas forcément optimale)</i> et de les étudier afin d'évaluer leurs résultats, en fonction de leur qualité et de leur temps d'exécution.

Nous avions le choix d'implémenter au minimum 2 des heuristiques proposé dans ce projet, nous avons finalement étudié les 4 heuristiques : 

- <u>Nearest Neighbor</u>, heuristique gloutonne qui nous servira pour d'autres méthodes comme solution initiale
- <u>Cheapest Insertion</u>, qui apporte une approche plus détaillée que Nearest Neighbor mais reste une heuristique gloutonne
- <u>Two-Opt</u>, heuristique de recherche locale se basant sur des 2-permutations
- <u>Simulated Annealing</u>, s'inspirant de la mettalurgie et apportant une part d'aléatoire pour améliorer Two-Opt


<br>

## <u>Fonctions utiles</u>

Voici les fonctions auxilliaires que nous avons utilisé à la réalisation de ce projet pour la bonne implémentation des heuristiques.

### distance.py

Permet de calculer la distance totale parcourue d'un chemin issue d'une heuristique.

In [ ]:
def total_path_distance(path, distance_matrix):
    path_length = 0
    for i in range(len(path) - 1):         # On cumule les distances entre les villes i et i + 1                    
        path_length += distance_matrix[path[i]][path[i + 1]]
    path_length += distance_matrix[path[-1]][path[0]]
    return path_length

<u>Paramètres d'entrée :</u>
- $path$ = $[p_0, p_1, \dots, p_{n-1}]$, où p est l'indice d'une ville, représente un chemin solution d'une des heuristiques
- $distance\_matrix$ : Une matrice carrée (liste de liste) des distances $D$ où $D[i][j]$ représente la distance entre les sommets $i$ et $j$ (entre la ville $i$ et la ville $j$)

<u>Retour de la fonction :</u>
- $path\_length$ : distance totale parcourue selon le chemin en entrée et la matrice des distances

### generate.py

$\fbox{Génération des villes}$

Tout d'abord, avant de commencer il nous faut générer un ensemble de villes, chacune avec des coordonnées $(x,y)$ afin de modéliser notre problème du TSP :

In [ ]:
import numpy as np
import random

class City:
    
    def __init__(self, x, y, id):
        self.x = x
        self.y = y
        self.id = id

    def distance_to(self, other: 'City'):
        return np.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)   # Distance euclidienne entre deux villes en fonction des coordonnées

def distance_matrix(matrix_size):
    cities = [City(random.uniform(0, 100), random.uniform(0, 100), i) for i in range(matrix_size)] # Liste contenant n villes (selon matrix_size) avec des coordonnées aléatoires
    distance_matrix = np.zeros((len(cities), len(cities)))                                         # La matrice est remplie de 0 pour l'initialiser

    for i, city1 in enumerate(cities):
        for j, city2 in enumerate(cities):
            distance_matrix[i][j] = city1.distance_to(city2)                                       # Calcul des distances entre chaque ville

    # Retour des coordonnées des villes pour la visualisation
    coords = [(v.x, v.y) for v in cities]
    
    return distance_matrix, coords

$\fbox{Classe City}$

<u>Attributs de la classe</u>

- $x$ (float) : représente la coordonnée X de la ville
- $y$ (float) : représente la coordonnée Y de la ville
- $id$ (int) : identifiant de la ville

$\fbox{Méthodes}$

1.<u> Fonction "distance_to"</u>

Cette fonction calcule la distance euclidienne entre la ville actuelle et une autre ville <i>(other)</i>

2.<u> Fonction "distance_matrix"</u>

<u>Paramètre d'entrée :</u>

- $matrix\_size$ : représente le nombre de villes à générer

<u>Retour de la fonction :</u> 

- $distance\_matrix$ : est une matrice carrée contenant les distances euclidiennes entre chaque paire de villes
- $coords$ : est une liste des coordonnées $(x,y)$ des villes générées, qui sera notamment utilisée pour la visualisation des résultats. 

### quality.py

$\fbox{Qualité des résultats}$

L'objectif de notre projet étant d'évaluer la qualité et la rapidité de chacunes des heuristiques implémentées, nous avons ci-dessous deux fonctions utilisées dans notre évaluation : 

In [ ]:
import functions.distance as distance
import time

def gap(total_path_distance, lower_bound):
    return f"{(total_path_distance - lower_bound) / lower_bound * 100} %"   # Calcule l'écart relatif entre la distance trouvée et la borne inférieure en pourcentage

def evaluate_quality(name, algo, distance_matrix, lower_bound):
    start_time = time.process_time()    # Début du chronomètre
    
    path = distance.total_path_distance(algo, distance_matrix)
    print(f'Route la plus courte pour l\'algorithme " {name} " : \n{algo}')
    print(f"Longueur totale du chemin trouvé : {path}")
    print(f"Qualité de la solution par rapport à la borne inférieure (Nearest Neighbor) : {gap(path, lower_bound)} de distance")
    
    end_time = time.process_time()      # Fin du chronomètre
    print(f"Temps de calcul : {end_time} s\n")

$\fbox{Fonction "gap"}$

<u>Paramètres d'entrées</u> :

- $total\_path\_distance$ : représente la distance totale du chemin proposé par l'algorithme
- $lower\_bound$ : borne inférieur servant de référence (ici résultat du Nearest Neighbor)

<u>Retour de la fonction</u> :

- $str$ : est le pourcentage d'écart par rapport à la borne inférieur $lower\_bound$

<u>Principe</u>

Cette fonction calcule l'écart relatif, en pourcentage, entre une solution trouvée par un algorithme et une borne inférieure, donc une valeur de référence minimale. Cette valeur de référence a été fixée au chemin obtenu par Nearest Neighbor. Cela nous permet de mesurer la qualité d'une solution "approximative".

$\fbox{Fonction "evaluate\_quality"}$

<u>Paramètres d'entrées</u>:

- $name$ (str) : représente le nom de l'algorithme
- $algo$ (list) : est une solution (liste ordonnée des villes parcourue) produite par l'algorithme à évaluer
- $distance\_matrix$ : matrice des distances entre les villes
- $lower\_bounds$ : borne inférieur pour évaluer la qualité de la solution


<u>Principe</u> :

- Lance un chronomètre avec $time.process\_time()$ pour mesurer uniquement le temps de calcul 
- Calcule la distance totale du chemin fourni
- Affiche le chemin, la distance totale et la qualité de la solution via $gap()$
- Affiche enfin le temps de calcul écoulé

### visualisation.py

$\fbox{Visualisation des résultats}$

Afin de bien se représenter commment chaque fonction implémentée construit une solution pour le TSP, nous avons choisi de créer une fonction de visualisation des résultats, sur un même tableau, dont le code est exposé ci-dessous :

In [ ]:
import matplotlib.pyplot as plt

def plot_path(coord, paths, names):
    """
    Visualisation des chemins obtenus avec :
        coord = Liste des coordonnées
        path = Ordre des villes visitées 
        name = nom de la technique utilisée
    
    """
    fig, axis = plt.subplots(1, len(paths), figsize=(5 * len(paths), 6))
    
    if len(paths) == 1: #Si un seul chemin: objet unique
        axis = [axis]

    for ax, path, name in zip(axis, paths, names):
        path_coords = [coord[i] for i in path] + [coord[path[0]]]
        
        # Séparation des coordonnées x et y pour créer deux listes avec chacune d'entre elles
        x_path, y_path = zip(*path_coords)
        
        # --- VISUALISATION DU CHEMIN OBTENU ---
        ax.plot(x_path, y_path, 'ro-', label ='Chemin TSP', markersize=5) #r --> rouge ; o --> marqueur cercle ; "-" --> relié par des lignes 
        
        # --- AFFICHAGE DES NUMEROS DE VILLES --
        # Parcours des coordonnées (x,y) originales pour associer à chaque numéros des villes 'i' avec un décalage pour la lisibilité
        for i, (x,y) in enumerate(coord):
            #Affiche le numéro de la ville 'i' (on décale un peu pour la lisibilité)
            ax.text(x,y + 1.5, str(i), color='blue', fontsize=12, ha='center') # ha=center --> Aligner horizontalement les numéros de villes aux cercles

        # Mise en évidence du point de départ
        start_city_coords = coord[path[0]]
        ax.plot(start_city_coords[0], start_city_coords[1], 'go', markersize=10, label="Ville de départ") #g --> vert ; o --> marqueur cercle


        # --- MISE EN FORME DU GRAPHIQUE ---
        ax.set_title(f"Méthode {name}")
        ax.set_xlabel("Coordonnée x")       #Axe des x
        ax.set_ylabel("Coordonnée y")       #Axe des y
        ax.legend()
        ax.grid(True)                       #Grille pour permettre une meilleure visualisation des chemins
        ax.axis('equal')                    #Evite les distorsions des axes x et y 
    
    plt.tight_layout()
    plt.show()                              #Affichage de la fenêtre

<u>Paramètres d'entrée :</u>

- $coord$ représentant les coordonnées $(x,y)$ de chaque ville $v$
- $paths$ est une liste de chemins dont chaque chemin est une liste d'indice représentant l'ordre des villes visitées selon chaque algorithme
- $names$ représente la liste des noms des méthodes utilisées pour générer les chemins de $paths$ (donc Nearest Neighbor, 2-opt, Simulated Annealing et Cheapest Insertion)

Pour chaque chemin, cette fonction trace les circuits TSP avec des lignes rouge reliant chaque ville, ensuite affiche les indices des villes en bleu et chacun d'entre elle sont représenté par un point bleu. La ville de départ est marquée en vert.

Plusieurs techniques sont utilisées pour optimiser l'algorithme et le rendu final :
- Nous égalisons les axes pour éviter certaines distorsions de distances
- Les titres, légendes et axes propres sont ajoutés à chaque sous-graphes
- Si nous avons qu'un chemin, nous traitons le cas seulement à travers la ligne :

$if$ $len(paths == 1 : axis = [axis])$

- Nous décompressons les coordonnées à l'aide de :

$zip(*path\_coords)$

Ainsi pour chaque chemin fourni nous : 

- Créerons une sous-figure distincte, pour afficher le chemin associé à telle méthode
- Traçons le chemin, en reliant les villes dans l'ordre donné par $path$
- Affichons les numéros de villes pour chaque point
- Mettons en évidence la ville de départ
- Enfin nous ajoutons les éléments visuels des graphiques comme les titres, légendes, axes etc...

<br>

## <u>Implémentation des Heuristiques</u>

Voici les 4 heuristiques implémentées ainsi que leur description et fonctionnement.

### Nearest Neighbor (neighbor.py)

$\fbox{Plus Proche Voisin}$

Le plus proche voisin est une heuristique de résolution du TSP dite "gloutonne". Cette approche a l'avantage de <strong>s'exécuter rapidement</strong> néanmoins elle ne garantit pas une solution optimale. Cette méthode produit un chemin qui s'avère utile en tant que solution initiale, notamment dans deux autres de nos heuristiques : Two-Opt et Simulated Annealing

In [ ]:
def nearest_neighbor(distance_matrix):
    """
    MÉTHODE DU VOISIN LE PLUS PROCHE : 
    Approche heuristique simple pour résoudre le TSP.
    Consiste tout simplement à chercher la ville la plus proche de la ville d'origine et ainsi de suite.
    
    Très rapide (mais pas du tout optimal) et permet d'offrir une base sur laquelle on peut apporter des améliorations (2-opt, simulated annealing)
    """
    n = len(distance_matrix)
    unvisited = set(range(n))   # Liste des villes non visitées
    current = 0
    path = [current]
    unvisited.remove(current)

    while unvisited:
        nearest = min(unvisited, key=lambda city: distance_matrix[current][city])   # On cherche la ville la plus proche
        path.append(nearest)                                                        # On l'ajoute au chemin final
        current = nearest
        unvisited.remove(nearest)                                                   # On enlève la ville la plus proche de unvisited

    path.append(path[0])                                                            # On ajoute la ville 0 pour compléter la boucle
    return path

<u>Paramètres en entrée:</u>
- $distance\_matrix$ est une matrice carrée contenant les distances entre chaque paire de villes. $distance\_matrix[i][j]$ représente donc la distance entre la ville i et la ville j

<u>Paramètres du bloc :</u>
- $n$ : Représente le nombre de villes
- $unvisited$ : Une liste représentant l'ensemble des villes non visitées
- $current$ : Représentant la ville actuelle 

<u>Retour de l'algorithme :</u>
- $path$ : une liste représentant l'ordre des villes à visiter, qui inclue un retour à la ville de départ (la ville 0) grâce à cette commande : $path.append(0)$

<u>Principe de l'algorithme :</u>
- Avant notre boucle $while$, nous ajoutons à notre chemin $path$ la ville actuelle ($current$), donc la ville de départ et naturellement, celle-ci est retirée de la liste $unvisited$
- Nous partons de la ville de départ (donc ici la ville 0)
- A chaque étape, l'algorithme choisit la ville non visitée la plus proche et l'ajoute à $path$ tout en la retirant de la liste des villes non visitées $unvisited$
- Ce processus est répété jusqu'à ce que toutes les villes aient été visitées 
- Se termine en revenant à la ville de départ, notamment en ajoutant celle-ci à la fin de $path$

### Cheapest Insertion (insertion.py)

$\fbox{Insertion Minimale}$

L'insertion minimale est une <strong>heuristique gloutonne</strong> qui aide à construire une solution initiale, souvent plus optimale que celle du Nearest Neighbor, au TSP. L'idée est de construire progressivement un chemin, en insérant à chaque tour la ville non encore visitée, en s'assurant que cet ajout augmente le moins possible le coût total du chemin.


In [ ]:
import functions.distance as distance

def cheapest_insertion(distance_matrix):
    """
    METHODE D'INSERTION :
    Construit une solution approchée du TSP par heuristique d'insertion

    La fonction prend en entrée une matrice carrée des distances entre les villes et
    renvoie un ordre de visite minimisant approximativment la distance totale du chemin

    """
    n = len(distance_matrix)                             #Nombre total des villes
    unvisited = list(range(n))                           #Liste des indices des villes non inserees dans le circuit
    path = [unvisited.pop(0), unvisited.pop(0)]          #Initialisation du chemin avec les deux premières villes


    while unvisited:
        #Recherche la meilleure ville à inserer et de sa position optimale

        best_cost = float('inf')                            #Init avec la valeur infini pour la technique de borne inférieure
        best_place = None
        to_insert = None                                    #Ville à insérer lorsque celle-ci a la plus faible augmentation du coût total

        for v in unvisited:                                 #Insertion de chaque ville restantes une à une
            for i in range(len(path)):
                test_path = path[:i+1] + [v] + path[i+1:]   #Création d'un nouveau chemin hypothétique avec la ville v
                new_cost = distance.total_path_distance(test_path, distance_matrix)

                #Si l'insertion donne un cout plus optimal alors conservation en mémoire comme meilleure option
                if new_cost < best_cost:
                    best_cost = new_cost    
                    best_place = i+1
                    to_insert = v

        #Insertion de la "meilleure ville" à la "meilleure position"
        path.insert(best_place, to_insert)
        unvisited.remove(to_insert)

    path.append(path[0])  #Ajout de la première ville à la fin pour fermer le chemin
    return path

<u>Paramètres d'entrée :</u>

- $distance\_matrix$ qui représente une matrice carrée $D$ des distance, comme celle évoquée pour $2-opt$ (voir la suite). 

<u>Retour de l'algorithme :</u> nous obtiendrons un circuit complet <strong>fermé</strong> (avec une liste d'indices de villes) qui commence et termine à la même ville de départ.

<u>Principe de l'algorithme :</u>

Tout d'abord on initialise $path$ avec les deux premières villes (0 et 1) et la liste $unvisited$ qui contient les villes restantes. 

Ensuite tant qu'il reste des villes qui n'ont pas été visitées pour chaque ville $v$ :

- On va tester toutes les positions d'insertion possibles dans le chemin actuel
- On calcule le coût total du chemin avec $path\_length\_matrix$ lorsque $v$ est supposément inséré à cette position.
- On garde l'insertion qui produit le coût total

Enfin on va insérer la meilleure ville trouvée à la meilleure position

L'algorithme s'arrête lorsque toutes les villes sont dans le chemin.

### 2-opt (twoopt.py)

$\fbox{Algorithme 2-OPT}$


L'algorihtme 2-opt est une méthode <strong>d'optimisation locale</strong> visant à améliorer une solution initiale, comme mentionné au-dessus. L'idée principale consiste à supprimer deux arêtes du chemin et ensuite à reconnecter les segments d'une autre manière, via des <strong>2-permutations</strong>, si cela permet la réduction du coût total du parcours. 

In [ ]:
def two_opt(path, distance_matrix):
    """
    METHODE 2-OPT : Amélioration d'un chemin du TSP
    
    Prend en entrée un chemin (une liste d'indices) et une matrice de distance
    Renvoie un chemin amélioré à l'aide de 2-permutations (inversion d'arête)
    """
    n = len(path)
    improve = True              #Initialisation du booléen d'amélioration à Vrai

    max_iterations = 1000       #Maximum d'itérations avant arrêt de l'algorithme
    iteration = 0               #Comptage d'itérations

    while improve and iteration < max_iterations:         #Evite que le calcul ne boucle trop longtemps notamment lorsque la solution intiale est de mauvaise qualité
        improve = False
        iteration += 1
    
        for i in range(n-1):
                for j in range(i+2, n if i>0 else n-1):   #Empêche les inversions invalides
                    a, b = path[i], path[i+1]             # a --> b
                    c, d = path[j], path[(j+1)%n]         # c --> d
                    
                    #Calcul du gain de distance
                    delta = -distance_matrix[a][b] -distance_matrix[c][d] + distance_matrix[a][c] + distance_matrix[b][d]

                    if delta < 0 :                       #Si delta est négatif, il y a un gain donc on procède à l'échange des arêtes
                        path[i+1:j+1] = reversed(path[i+1:j+1])
                        improve = True

        if iteration >= max_iterations :                 #Condition permettant d'éviter de rallonger le calcul lorsque la solution initale n'est pas du tout optimale 
             print(f"AVERTISSEMENT : Arret apres {iteration} iterations, limite maximum atteinte\n")
    return path

<u>Paramètres d'entrée :</u>

- $path$ un chemin = $[p_0, p_1, \dots, p_{n-1}]$
- Une matrice des distances $D$ où $D[i][j]$ donne la distance entre les sommets $i$ et $j$.

Et nous allons effectuer nos calculs sur deux arêtes $(a,b) = (p_i, p_{i+1})$ et $(c, d) = (p_j, p_{j+1})$ et on propose de remplacer ces deux arêtes par $(a,c)$ et $(b,d)$, en inversant la sous partie entre $b$ et $c$.


<strong>Calcul du gain de coût</strong>

Le point central est le <strong>calcul du gain</strong> par l'intermédiaire du coefficient $\Delta$, calculé comme suivant : 

$\Delta = -D[p_i][p_{i+1}] - D[p_j][p_{(j+1) \bmod n}] + D[p_i][p_j] + D[p_{i+1}][p_{(j+1) \bmod n}]$

Si $\Delta < 0$ alors le coût du chemin est plus bénéfique et donc nous inversons la sous-séquence $path[i+1 : j+1]$


<strong>Conditions d'arrêt</strong>
- STAGNATION : Une première condition d'arrêt est simplement lorsque l'algorithme ne voit plus de possibilité d'améliorer et d'effectuer des 2-permutions. Alors dans ce cas, le chemin obtenu est considéré comme amélioré au mieux grâce à 2-opt

- LIMITE : Cette seconde condition a été développée à la suite de soucis de boucles longues et inefficaces, surtout lorsque la solution initiale entrée est mauvais. 

    $max\_iterations = 1000$

Si le seuil est atteint, le processus est interrompu, même si nous n'avons pas encore atteint la stagnation.

### Simulated Annealing (annealing.py)

$\fbox{Algorithme du Recuit Simulé}$

Permet, en acceptant de manière probabiliste des solutions de moins bonne qualité, d'éviter les minimas locaux dans la recherche. Utilise les inversions du 2-Opt pour améliorer la solution, compare le delta (variation des distances) avec une probabilité dépendant de la température, et intègre un schéma de refroidissement qui réduit progressivement et lentement la température à chaque itération. 

In [ ]:
import math
import random

def simulated_annealing(distance_matrix, path, lower_bound, initial_temperature, final_temperature, cooling_rate):
    """
    MÉTHODE DU RECUIT SIMULÉ :
    Technique inspirée de la métallurgie acceptant temporairement des solutions moins bonnes pour éviter les minimas locaux.
    Implique l'aléatoire et une diminution de la température au fil des itérations puis se base sur le 2-opt pour la permutation des arêtes.
    """
    n = len(path)
    best_length = lower_bound
    best_path = path
    T = initial_temperature
    iteration = 0
    while T > final_temperature:
        i = random.randint(0, n-1)              # La première ville est choisie au hasard
        j = (i + random.randint(2, n-2)) % n    # La deuxième ville est également choisie au hasard et forcément différente de la première
        if j < i:
            i, j = j, i                         # j est toujours après i dans l'ordre du chemin
            
        delta = distance_matrix[path[i]][path[j]] + distance_matrix[path[i+1]][path[(j+1) % n]]\
                - distance_matrix[path[i]][path[i + 1]] - distance_matrix[path[j]][path[(j + 1) % n]] # Calcule la variation de la longueur totale du chemin s'il y a une inversion
                
        if delta < 0 or math.exp(-delta / T) > random.random():             # Si delta négatif, amélioration donc on accepte, sinon on accepte en fonction de delta et de la température
            lower_bound = lower_bound + delta
            for k in range((j - i) // 2):
                path[k + i + 1], path[j - k] = path[j - k], path[k + i + 1] # Inversion d'arête venant du 2-opt

        # Vérifie s'il y a une amélioration de la solution
        if best_length > lower_bound:
            best_length = lower_bound
            best_path = path
        iteration += 1
        if iteration % (n * n) == 0:
            T *= cooling_rate # Réduction de la température

    return best_path, best_length

<u>Paramètres d'entrée :</u>
- $distance\_matrix$ : Matrice des distances entre les villes
- $path$ : Chemin initial (celui issue de Nearest Neighbor)
- $lower\_bound$ : La distance totale du chemin initial (On se sert de Nearest Neighbor comme borne inférieure)
- $initial\_temperature$ : Température initiale, elle doit être élevée pour une recherche de solution large
- $final\_temperature$ : Température finale, seuil d'arrêt de l'algorithme, elle doit être faible pour une bonne précision de la solution
- $cooling\_rate$ : Facteur de refroidissement (< 1) pour permettre la réduction de la température à chaque itération

<u>Principe de l'algorithme :</u>
- La boucle principale $while$ continue tant que la température reste supérieure à la température finale
- Une première ville est sélectionnée au hasard, la deuxième l'est également mais on s'assure qu'elle est "plus loin" dans le chemin.
- Calcul du delta c'est-à-dire la variation de la longueur du chemin si le mouvement 2-opt est appliqué. On prend les distances entre les villes avant inversion et on le soustrait aux distances après inversion
- Si le delta est négatif, le nouveau chemin est meilleur et on l'accepte. Sinon on l'accepte avec une probabilité donnée par $exp(-delta/T)$. On effectue alors inversion du 2-opt.
- On met enfin à jour la solution si elle est meilleure et on refroidit périodiquement et progressivement la température. À la fin le meilleur chemin et la meilleure longueur sont trouvés.

<br>

## <u>Main</u>

In [ ]:
"""
PROJET MATHS-INFO
"""

### IMPORTS ###
import functions.generate as generate
import algorithms.neighbor as neighbor
import algorithms.insertion as insertion
import algorithms.twoopt as twoopt
import algorithms.annealing as annealing
import functions.visualisation as vs
import functions.quality as qc
import functions.distance as distance

### MAIN ###
distance_matrix, coords = generate.distance_matrix(50)

print("Matrice des distances entre les villes: \n")
print(distance_matrix)
print("\n")
print("Note : si le pourcentage est négatif il y a amélioration, si le pourcentage est positif alors il y a dégradation par rapport à Nearest Neighbor.")
    
algo1 = neighbor.nearest_neighbor(distance_matrix)
lower_bound = distance.total_path_distance(algo1, distance_matrix) # Référence pour les autres algorithmes
qc.evaluate_quality("Nearest Neighbor", algo1, distance_matrix, lower_bound)

#Copie du chemin obtenu avec Nearest Neighbor, pour TwoOpt et Simulated Annealing
path = algo1.copy()

algo2 = insertion.cheapest_insertion(distance_matrix)
qc.evaluate_quality("Cheapest Insertion", algo2, distance_matrix, lower_bound)

# Choix du chemin de Nearest Neighbor pour le calcul des deux prochaines heuristiques
algo3 = twoopt.two_opt(path, distance_matrix)
qc.evaluate_quality("2-Opt", algo3, distance_matrix, lower_bound)

# Paramètres de Simulated Annealing
initial_temperature = 5000
cooling_rate = 0.995
path = algo1.copy()
path.pop() # On enlève le dernier 0 de Nearest Neighbor pour éviter les doublons

algo4_path, algo4_length = annealing.simulated_annealing(distance_matrix, path, lower_bound, initial_temperature, 0.001, cooling_rate)
qc.evaluate_quality("Simulated Annealing", algo4_path, distance_matrix, lower_bound)

# Affichage sur la même figure des visualisations des 4 heuristiques
vs.plot_path(coords, [algo1, algo2, algo3, algo4_path], ["Nearest Neighbor", "Insertion", "Two Opt", "Simulated Annealing"])